In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Endometriosis"
cohort = "GSE120103"

# Input paths
in_trait_dir = "../../input/GEO/Endometriosis"
in_cohort_dir = "../../input/GEO/Endometriosis/GSE120103"

# Output paths
out_data_file = "../../output/preprocess/Endometriosis/GSE120103.csv"
out_gene_data_file = "../../output/preprocess/Endometriosis/gene_data/GSE120103.csv"
out_clinical_data_file = "../../output/preprocess/Endometriosis/clinical_data/GSE120103.csv"
json_path = "../../output/preprocess/Endometriosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Homo sapiens whole genome expression microarray of endometrium obtained from fertile and Infertile women with stage IV ovarian endometriosis and without endometriosis"
!Series_summary	"The hypothesis that male michrochimerism in eutopic endometrium is a factor for endometriosis, as indicated by indirect evidence was examined in endometrial samples from control (Group 1) and stage IV ovarian endometriosis (Group 2), either fertile (Group 1A and 2A) or Infertile (Group 1B and 2B) pateints."
!Series_summary	"6 coding and 10 non-coding genes showed bi-modal pattern of expression characterised by low expression in samples obtained from fertile patients and high expressions in infertile patients. Several coding and non-coding MSY-linked genes displayed michrochimerism in form of presence of their respective DNA inserts along with their microarray-detectable expression in endometrium irrespective of fertility history and disease."
!Series_overall_design	

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the series description, this appears to be a whole genome expression microarray dataset,
# so we can confidently set this to True
is_gene_available = True

# 2.1 Data Availability
# For the trait (endometriosis), the relevant key is 1, which indicates sample group information
trait_row = 1  

# For age, there's no information available in the sample characteristics
age_row = None

# For gender, we can see that gender information is available at key 0,
# but it shows only one value "Female", so it's a constant feature and should be considered unavailable
gender_row = None  

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert sample group to binary values for endometriosis trait."""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Based on the values, we need to determine who has endometriosis
    if "Stage IV Ovarian Endometriosis" in value:
        return 1  # Has endometriosis
    elif "Disease free Endometrium" in value:
        return 0  # Does not have endometriosis
    else:
        return None

def convert_age(value):
    """Convert age to continuous values."""
    # This function is included for completeness but won't be used as age data is not available
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender to binary values."""
    # This function is included for completeness but won't be used as gender data is not available
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if value == "female":
        return 0
    elif value == "male":
        return 1
    else:
        return None

# 3. Save Metadata
# Determine if trait data is available based on whether trait_row is None
is_trait_available = trait_row is not None

# Call the validation function
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a proper DataFrame for the clinical data
    # Using a more appropriate structure for geo_select_clinical_features
    
    # Initialize sample characteristics dictionary with proper structure
    sample_characteristics = {
        0: ['gender: Female'], 
        1: ['sample group: Disease free Endometrium of fertile women', 
            'sample group: Endometrium  from Stage IV Ovarian Endometriosis of fertile women', 
            'sample group: Disease free Endometrium of Infertile women', 
            'sample group: Endometrium  from Stage IV Ovarian Endometriosis of Infertile women'], 
        2: ['tissue: Endometrium']
    }
    
    # Create a clinical DataFrame with appropriate format for geo_select_clinical_features
    # Each row represents one characteristic (like trait, gender)
    clinical_data_dict = {}
    for key, values in sample_characteristics.items():
        clinical_data_dict[key] = values
    
    clinical_data = pd.DataFrame.from_dict(clinical_data_dict, orient='index')
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the results
    preview = preview_df(selected_clinical_df)
    print("Clinical Features Preview:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical features to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")


Clinical Features Preview:
{0: [0.0], 1: [1.0], 2: [0.0], 3: [1.0]}
Clinical features saved to ../../output/preprocess/Endometriosis/clinical_data/GSE120103.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 66
Header line: "ID_REF"	"GSM3393491"	"GSM3393492"	"GSM3393493"	"GSM3393494"	"GSM3393495"	"GSM3393496"	"GSM3393497"	"GSM3393498"	"GSM3393499"	"GSM3393500"	"GSM3393501"	"GSM3393502"	"GSM3393503"	"GSM3393504"	"GSM3393505"	"GSM3393506"	"GSM3393507"	"GSM3393508"	"GSM3393509"	"GSM3393510"	"GSM3393511"	"GSM3393512"	"GSM3393513"	"GSM3393514"	"GSM3393515"	"GSM3393516"	"GSM3393517"	"GSM3393518"	"GSM3393519"	"GSM3393520"	"GSM3393521"	"GSM3393522"	"GSM3393523"	"GSM3393524"	"GSM3393525"	"GSM3393526"
First data line: "(+)E1A_r60_1"	0.2933545	0.012980461	-0.23412466	-0.7159295	-0.012980461	-1.7718949	-2.2321372	-1.1389613	0.4525237	-1.8300767	-2.9251266	-2.3127236	-1.1785226	-2.9045892	-0.2190404	-1.5515742	-2.569182	-1.8102136	14.862822	14.86257	14.874119	14.430498	14.441908	14.338968	14.85437	-2.4582553	-2.5016837	-0.20707464	0.17501879	0.013408184	0.5477576	6.3862505	5.2938547	6.3140154	0.4224906	3.351428
Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_

### Step 4: Gene Identifier Review

In [5]:
# Review gene identifiers in the data
# The identifiers such as 'A_23_P100001', 'A_23_P100011', etc. appear to be Agilent microarray probe IDs
# rather than standard human gene symbols. These will need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan]

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns from the gene annotation data
# From the preview, we can see that 'ID' contains the probe identifiers (like A_23_P100001) 
# which match the gene expression data index, and 'GENE_SYMBOL' contains the gene symbols

# 2. Get gene mapping dataframe by extracting the ID and GENE_SYMBOL columns
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

# Print preview of the mapping dataframe
print("Gene mapping preview:")
print(preview_df(mapping_df))

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# This handles the many-to-many relationship between probes and genes
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print the shape of the resulting gene expression dataframe
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10])

# Preview the gene expression data
print("Gene expression data preview:")
print(preview_df(gene_data))


Gene mapping preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'Gene': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN']}
Gene expression data shape after mapping: (17678, 36)
First few gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT',
       'AAAS', 'AACS'],
      dtype='object', name='Gene')
Gene expression data preview:
{'GSM3393491': [-1.3306704, -1.0435967, 1.993511226, 2.275426, 0.53070736], 'GSM3393492': [-1.4305425, -0.54736996, 0.54174758, 0.47634304, 0.3492117], 'GSM3393493': [-1.7391768, -0.5067458, 2.64482165, 0.9172963, -0.19257355], 'GSM3393494': [-0.65010595, 0.27574682, -1.99191613, 1.6102637, 0.24081898], 'GSM3393495': [-0.7661872, -1.1883221, 0.8087396800000001, 2.1443443, 0.3116703], 'GSM3393496': [0.43882227, -0.002605438, -1.50206663, -0.15116894, 0.49943542], 'GSM3393497': [0.25899267, -0.015143395, -2.4413662, -0.30618966, 0.19794655], 'GSM3393498': [0.39300776, 

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Read the clinical data that was saved earlier
print("Reading previously saved clinical data")
# First check if the clinical data file exists
if os.path.exists(out_clinical_data_file):
    clinical_features_df = pd.read_csv(out_clinical_data_file)
    print("Clinical data preview:")
    print(clinical_features_df.head())
else:
    # If file doesn't exist, extract clinical features again
    print("Clinical data file not found, extracting clinical features again")
    clinical_features_df = geo_select_clinical_features(
        clinical_data, 
        trait=trait, 
        trait_row=trait_row, 
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features_df.to_csv(out_clinical_data_file)

# Transpose the clinical_features_df to have samples as rows and features as columns
# This is necessary because the current format has features as rows, which is not what we want for linking
clinical_features_df_transposed = clinical_features_df.T
clinical_features_df_transposed = clinical_features_df_transposed.rename(columns={0: trait})
print("Transposed clinical features shape:", clinical_features_df_transposed.shape)
print("Transposed clinical features columns:", clinical_features_df_transposed.columns)

# Now link the clinical and genetic data
linked_data = normalized_gene_data.T
linked_data = pd.concat([clinical_features_df_transposed, linked_data], axis=1)
print("Linked data shape:", linked_data.shape)
print("Linked data columns preview:", linked_data.columns[:10])

# Check if trait column exists in linked data
if trait in linked_data.columns:
    print(f"Trait column '{trait}' found in linked data")
    print(linked_data[trait].value_counts())
else:
    print(f"Warning: '{trait}' column not found in linked data")
    # If trait column is missing, we have a problem
    raise ValueError(f"The trait column '{trait}' is missing in the linked data")

# Handle missing values in the linked data
linked_data_cleaned = handle_missing_values(linked_data, trait)
print("Linked data shape after cleaning:", linked_data_cleaned.shape)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from endometrium of women with and without stage IV ovarian endometriosis, including both fertile and infertile subjects."
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Endometriosis/gene_data/GSE120103.csv
Reading previously saved clinical data
Clinical data preview:
     0    1    2    3
0  0.0  1.0  0.0  1.0
Transposed clinical features shape: (4, 1)
Transposed clinical features columns: Index(['Endometriosis'], dtype='object')
Linked data shape: (40, 17450)
Linked data columns preview: Index(['Endometriosis', 'A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A4GALT',
       'A4GNT', 'AAAS', 'AACS'],
      dtype='object')
Trait column 'Endometriosis' found in linked data
Endometriosis
0.0    2
1.0    2
Name: count, dtype: int64
Linked data shape after cleaning: (0, 1)
Quartiles for 'Endometriosis':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Endometriosis' in this dataset is fine.

Abnormality detected in the cohort: GSE120103. Preprocessing failed.
Data was determined to be unusable and was not saved
